## Prep

In [1]:
import pandas as pd

import datetime as dt
import os
import io
from github import Github
import logging

import Universes
import utils_for_alpaca

In [2]:

import alpaca
from alpaca.common.enums import Sort
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass, AssetStatus, AssetExchange, OrderStatus, OrderType, OrderClass, QueryOrderStatus, CorporateActionType, CorporateActionSubType
from alpaca.trading.requests import GetCalendarRequest, GetAssetsRequest, GetOrdersRequest, MarketOrderRequest, LimitOrderRequest, StopLossRequest, TakeProfitRequest, TrailingStopOrderRequest, ReplaceOrderRequest, GetPortfolioHistoryRequest, GetCorporateAnnouncementsRequest
from alpaca.data.requests import StockLatestQuoteRequest, StockTradesRequest, StockLatestTradeRequest, StockQuotesRequest, StockBarsRequest, StockSnapshotRequest, StockLatestBarRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.enums import Adjustment, DataFeed, Exchange
from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.broker.client import BrokerClient

In [2]:
alpaca = utils_for_alpaca.MyAlpaca(os.environ['ALPACA_API_KEY_PAPER'],os.environ['ALPACA_API_SECRET_PAPER'])

2024-07-26 01:28:52,893 - root - INFO - Successfully read config for PaperTesting strategy
2024-07-26 01:28:53,107 - root - INFO - Strategy PaperTesting has been initialized


In [3]:
API_KEY_PAPER = os.environ['ALPACA_API_KEY_PAPER']
API_SECRET_PAPER = os.environ['ALPACA_API_SECRET_PAPER']
stock_client = StockHistoricalDataClient(API_KEY_PAPER,API_SECRET_PAPER)
broker_client = BrokerClient(API_KEY_PAPER,API_SECRET_PAPER,sandbox=False,api_version="v2")
trading_client = TradingClient(API_KEY_PAPER, API_SECRET_PAPER) # dir(trading_client)

## Get Orders

In [ ]:
date_filter = (pd.Timestamp.now()- pd.Timedelta(30, "days")) # orders for last 30 days
request_params = GetOrdersRequest(status=QueryOrderStatus.CLOSED, after = date_filter)
orders_data = trading_client.get_orders(filter=request_params)
orders_dicts = []
for entry in orders_data:
    entry_dict = {}  # Create a new dictionary for each entry
    for attribute, value in vars(entry).items():
        entry_dict[attribute] = value  # Populate the dictionary
    orders_dicts.append(entry_dict)  # Append the populated dictionary to the list
orders = pd.DataFrame(orders_dicts)
columns_to_convert = ['created_at','updated_at','submitted_at','filled_at']
for col in columns_to_convert:
    orders[col] = orders[col].dt.tz_convert('Europe/Berlin')
    orders[col] = orders[col].dt.strftime('%Y-%m-%d %H:%M')
columns_to_convert_float = ['trail_price', 'hwm', 'trail_percent', 'stop_price', 'limit_price', 'filled_avg_price']
orders[columns_to_convert_float] = round(orders[columns_to_convert_float].astype(float),2)
columns_to_convert_integer = ['qty', 'filled_qty']
orders[columns_to_convert_integer] = orders[columns_to_convert_integer].astype(int)
orders['strategy'] = orders['client_order_id'].str.split('_').str[0]
needed_cols = ['symbol','strategy','client_order_id', 'created_at', 'updated_at', 'submitted_at', 'filled_at', 'filled_qty', 'filled_avg_price', 'order_class','side']
orders.to_excel('orders.xlsx', columns=needed_cols, index=None)

In [ ]:
positions = trading_client.get_all_positions()
position_dicts = []
for entry in positions:
    entry_dict = {}  # Create a new dictionary for each entry
    for attribute, value in vars(entry).items():
        entry_dict[attribute] = value  # Populate the dictionary
    position_dicts.append(entry_dict)  # Append the populated dictionary to the list
df_positions = pd.DataFrame(position_dicts)
cols_to_number = ['avg_entry_price','qty','market_value','cost_basis','unrealized_pl',
                    'unrealized_plpc','unrealized_intraday_pl','unrealized_intraday_plpc',
                    'current_price','lastday_price','change_today','qty_available']
for column in cols_to_number:
    df_positions[column] = pd.to_numeric(df_positions[column], errors='coerce')
df_positions['unrealized_plpc'] = (df_positions['unrealized_plpc'] * 100).astype(int)
cols_needed = ['symbol','avg_entry_price','current_price','qty','side','market_value','unrealized_plpc']
df_positions[cols_needed].sort_values(by='unrealized_plpc',ascending=False)

In [47]:
trading_client.get_open_position('NFLX').qty

'5'

## Strategies profitability

Orders for the last 90 days (max 500 orders)

In [4]:
days_of_stat = 90
date_filter = (pd.Timestamp.now()- pd.Timedelta(days_of_stat, "days")) # if it a stock in position, it has been bought with max_holding_time
request_params = GetOrdersRequest(status=QueryOrderStatus.CLOSED, after = date_filter, limit = 500)
orders_data = trading_client.get_orders(filter=request_params)
# some tech stuff as the orders_data are of strange type ----------------------------
orders_dicts = []
for entry in orders_data:
    entry_dict = {}  # Create a new dictionary for each entry
    for attribute, value in vars(entry).items():
        entry_dict[attribute] = value  # Populate the dictionary
    orders_dicts.append(entry_dict)  # Append the populated dictionary to the list
len(orders_dicts)

82

Getting current positions

In [ ]:
positions = trading_client.get_all_positions()
position_dicts = []
for entry in positions:
    entry_dict = {}  # Create a new dictionary for each entry
    for attribute, value in vars(entry).items():
        entry_dict[attribute] = value  # Populate the dictionary
    position_dicts.append(entry_dict)  # Append the populated dictionary to the list
current_positions = pd.DataFrame(position_dicts)
current_positions
cols_needed = ['symbol','qty','side','current_price']
current_positions = current_positions[cols_needed].copy()
current_positions['current_price'] = round(current_positions['current_price'].astype(float),2)
current_positions['qty'] = current_positions['qty'].astype(int)
current_positions

Orders as dataframe

In [ ]:
orders = pd.DataFrame(orders_dicts)
needed_cols = ['symbol','client_order_id', 'side','filled_at', 'filled_qty', 'filled_avg_price']
orders = orders[needed_cols].copy()
orders['filled_at'] = orders['filled_at'].dt.tz_convert('Europe/Berlin').dt.tz_localize(None)
# orders['filled_at'] = orders['filled_at'].dt.tz_convert('Europe/Berlin').dt.strftime('%Y-%m-%d %H:%M')
orders['filled_avg_price'] = round(orders['filled_avg_price'].astype(float),2)
orders['filled_qty'] = orders['filled_qty'].astype(int)
orders['strategy'] = orders['client_order_id'].str.split('_').str[0]
orders.loc[orders['strategy'].str[8] == '-', 'strategy'] = 'manual' # get rows where the 9th character in 'strategy' is "-" and update those rows to "manual"
# Sort the DataFrame by filled_at to ensure chronological order
orders.sort_values(by=['filled_at'], inplace=True)
orders

In [8]:
orders.to_excel('orders.xlsx')

In [7]:
from collections import deque

# Initialize dictionaries to hold the FIFO queue for each stock (long and short)
long_positions = {}
short_positions = {}

results = [] # Initialize a list to store the results

for index, row in orders.iterrows():
    symbol = row['symbol']
    side = row['side']
    filled_qty = row['filled_qty']
    filled_avg_price = row['filled_avg_price']
    filled_at = row['filled_at']
    strategy = row['strategy']
    
    if side == OrderSide.BUY:
        # Check if closing a short position
        if symbol in short_positions and short_positions[symbol]:
            qty_to_buy = filled_qty
            while qty_to_buy > 0 and short_positions[symbol]:
                sell_qty, sell_price, sell_filled_at, sell_strategy = short_positions[symbol].popleft()
                
                if sell_qty > qty_to_buy:
                    short_positions[symbol].appendleft((sell_qty - qty_to_buy, sell_price, sell_filled_at, sell_strategy))
                    results.append({
                        'symbol': symbol,
                        'filled_qty': qty_to_buy,
                        'buy_filled_at': filled_at,
                        'filled_avg_price_buy': filled_avg_price,
                        'filled_avg_price_sell': sell_price,
                        'profitability': (sell_price - filled_avg_price) * qty_to_buy,
                        'strategy': sell_strategy,
                        'days_in_portfolio': (filled_at - sell_filled_at).days,
                        'status': 'realized'
                    })
                    qty_to_buy = 0
                else:
                    results.append({
                        'symbol': symbol,
                        'filled_qty': sell_qty,
                        'buy_filled_at': filled_at,
                        'filled_avg_price_buy': filled_avg_price,
                        'filled_avg_price_sell': sell_price,
                        'profitability': (sell_price - filled_avg_price) * sell_qty,
                        'strategy': sell_strategy,
                        'days_in_portfolio': (filled_at - sell_filled_at).days,
                        'status': 'realized'
                    })
                    qty_to_buy -= sell_qty
        else:
            # Add the buy order to the FIFO queue for the stock
            if symbol not in long_positions:
                long_positions[symbol] = deque()
            long_positions[symbol].append((filled_qty, filled_avg_price, filled_at, strategy))

    elif side == OrderSide.SELL:
        # Check if closing a long position
        if symbol in long_positions and long_positions[symbol]:
            qty_to_sell = filled_qty
            while qty_to_sell > 0 and long_positions[symbol]:
                buy_qty, buy_price, buy_filled_at, buy_strategy = long_positions[symbol].popleft()
                
                if buy_qty > qty_to_sell:
                    long_positions[symbol].appendleft((buy_qty - qty_to_sell, buy_price, buy_filled_at, buy_strategy))
                    results.append({
                        'symbol': symbol,
                        'filled_qty': qty_to_sell,
                        'buy_filled_at': buy_filled_at,
                        'filled_avg_price_buy': buy_price,
                        'filled_avg_price_sell': filled_avg_price,
                        'profitability': (filled_avg_price - buy_price) * qty_to_sell,
                        'strategy': buy_strategy,
                        'days_in_portfolio': (filled_at - buy_filled_at).days,
                        'status': 'realized'
                    })
                    qty_to_sell = 0
                else:
                    results.append({
                        'symbol': symbol,
                        'filled_qty': buy_qty,
                        'buy_filled_at': buy_filled_at,
                        'filled_avg_price_buy': buy_price,
                        'filled_avg_price_sell': filled_avg_price,
                        'profitability': (filled_avg_price - buy_price) * buy_qty,
                        'strategy': buy_strategy,
                        'days_in_portfolio': (filled_at - buy_filled_at).days,
                        'status': 'realized'
                    })
                    qty_to_sell -= buy_qty            
        else:
            # Add the sell order to the FIFO queue for short positions
            if symbol not in short_positions:
                short_positions[symbol] = deque()
            short_positions[symbol].append((filled_qty, filled_avg_price, filled_at, strategy))

# Append remaining unsold stocks to results and use current_price for profitability calculation
for symbol, queue in long_positions.items():
    if symbol in current_positions['symbol'].values:
        current_price = current_positions[current_positions['symbol'] == symbol]['current_price'].values[0]
        while queue:
            buy_qty, buy_price, buy_filled_at, buy_strategy = queue.popleft()
            results.append({
                'symbol': symbol,
                'filled_qty': buy_qty,
                'buy_filled_at': buy_filled_at,
                'filled_avg_price_buy': buy_price,
                'filled_avg_price_sell': current_price,
                'profitability': (current_price - buy_price) * buy_qty,
                'strategy': buy_strategy,
                'days_in_portfolio': (dt.datetime.now() - buy_filled_at).days,
                'status': 'unrealized'
            })

for symbol, queue in short_positions.items():
    if symbol in current_positions['symbol'].values:
        current_price = current_positions[current_positions['symbol'] == symbol]['current_price'].values[0]
        while queue:
            sell_qty, sell_price, sell_filled_at, sell_strategy = queue.popleft()
            results.append({
                'symbol': symbol,
                'filled_qty': sell_qty,
                'buy_filled_at': dt.datetime.now(),
                'filled_avg_price_buy': current_price,
                'filled_avg_price_sell': sell_price,
                'profitability': (sell_price - current_price) * sell_qty,
                'strategy': sell_strategy,
                'days_in_portfolio': (dt.datetime.now() - sell_filled_at).days,
                'status': 'unrealized'
            })

# Convert results to DataFrame
result_df = pd.DataFrame(results)

In [ ]:
# result_df.to_excel('merged.xlsx')
result_df

In [17]:
result_df.groupby('strategy').agg(
    total_profit=('profitability', 'sum'),
    number_of_trades=('profitability', 'count'),
    average_profit=('profitability', 'mean')
).reset_index()

,strategy,total_profit,number_of_trades,average_profit
0,BuyStreaksAt11,34.07,44,0.774318
1,BuyTopPremarket,-1145.62,4,-286.405000
2,manual,3151.50,4,787.875000


## Relative volume

In [4]:
all_tickers = alpaca.get_strategy_universe()

2024-07-24 00:20:35,203 - root - INFO - Strategy PaperTesting has universe All (366 tickers)


In [5]:
df_6mohist = alpaca.get_history(symbols=all_tickers,periods=14*22*6, FrameLength=30, frame = 'min')
len(df_6mohist)

2024-07-24 00:20:40,170 - root - INFO - Starting concurrent download of min data across 366 tickers...
2024-07-24 00:23:33,527 - root - INFO - Done downloading min data. 464866 rows collected.


464866

In [6]:
df_6mohist['time_period'] = df_6mohist['timestamp'].dt.strftime('%H:%M:%S')
average_df  = df_6mohist.groupby(['symbol', 'time_period']).agg({'volume': 'mean', 'trade_count': 'mean'}).reset_index()
average_df

,symbol,time_period,volume,trade_count
0,AA,09:30:00,721971.296703,5869.186813
1,AA,10:00:00,457054.032967,4505.802198
2,AA,10:30:00,371500.934066,3779.747253
3,AA,11:00:00,315071.681319,3368.923077
4,AA,11:30:00,295574.428571,3085.450549
...,...,...,...,...
5119,ZIM,14:00:00,219929.593407,1595.703297
5120,ZIM,14:30:00,226045.846154,1615.934066
5121,ZIM,15:00:00,274941.472527,2005.439560
5122,ZIM,15:30:00,701293.956044,4625.890110


In [7]:
pivot_df = average_df.pivot_table(index='symbol', columns='time_period', values=['volume', 'trade_count'])

# Flatten the multi-level column index
pivot_df.columns = [f'{val}_{col.replace(":", "")}' for val, col in pivot_df.columns]

# Reset the index to turn 'symbol' back into a column
pivot_df = pivot_df.reset_index()
pivot_df

,symbol,trade_count_093000,trade_count_100000,trade_count_103000,trade_count_110000,trade_count_113000,trade_count_120000,trade_count_123000,trade_count_130000,trade_count_133000,...,volume_113000,volume_120000,volume_123000,volume_130000,volume_133000,volume_140000,volume_143000,volume_150000,volume_153000,volume_160000
0,AA,5869.186813,4505.802198,3779.747253,3368.923077,3085.450549,2639.230769,2540.769231,2464.747253,2455.901099,...,2.955744e+05,2.372674e+05,2.198084e+05,2.192937e+05,2.129271e+05,2.527825e+05,2.390727e+05,3.040565e+05,9.967445e+05,7.134325e+05
1,AAL,12097.109890,8717.725275,6884.406593,6408.879121,5890.296703,4790.164835,4537.373626,4490.923077,4364.450549,...,1.981287e+06,1.599542e+06,1.462085e+06,1.504081e+06,1.420485e+06,1.571932e+06,1.621758e+06,1.954537e+06,5.000707e+06,2.359442e+06
2,AAPL,96861.945055,67369.483516,99358.868132,92661.978022,56033.098901,34182.736264,29638.527473,28539.483516,28608.846154,...,3.093613e+06,2.768289e+06,2.591033e+06,2.555613e+06,2.490371e+06,2.732877e+06,2.658163e+06,3.107504e+06,7.262819e+06,1.211316e+07
3,ABBV,6920.131868,6292.780220,5110.087912,4375.439560,4303.021978,3796.923077,3641.285714,3532.406593,3696.549451,...,2.137208e+05,1.798931e+05,1.765009e+05,1.783869e+05,1.818613e+05,2.067664e+05,2.021418e+05,2.552314e+05,7.892670e+05,1.649634e+06
4,ABEV,1874.868132,1551.186813,1531.912088,1214.010989,1113.131868,1205.417582,1088.901099,1132.054945,1264.659341,...,9.054186e+05,9.539528e+05,8.925620e+05,9.947583e+05,9.174835e+05,1.057867e+06,1.005659e+06,1.122900e+06,2.288841e+06,4.471266e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,WYNN,2274.648352,2016.945055,1827.054945,1665.483516,1642.153846,1553.241758,1558.780220,1521.329670,1525.934066,...,7.365032e+04,7.230466e+04,7.291925e+04,7.046075e+04,6.857899e+04,7.925064e+04,8.473979e+04,1.011616e+05,3.564757e+05,2.444175e+05
362,X,2469.879121,2033.296703,1707.604396,1583.912088,1553.791209,1537.461538,1967.065934,2375.494505,1937.923077,...,1.408691e+05,1.489141e+05,2.136088e+05,3.146976e+05,2.148692e+05,2.236413e+05,2.197105e+05,2.672080e+05,6.355525e+05,2.982971e+05
363,XOM,19151.472527,14475.549451,11482.538462,9873.626374,9001.384615,8627.824176,7883.263736,7438.230769,7448.802198,...,7.168227e+05,6.788962e+05,6.294915e+05,5.858790e+05,5.704752e+05,6.296015e+05,6.572443e+05,7.968495e+05,2.654914e+06,3.824719e+06
364,ZETA,1895.472527,1609.824176,1469.593407,1303.769231,1208.835165,1222.054945,1171.263736,1134.175824,1100.933333,...,1.040250e+05,1.121270e+05,1.027822e+05,9.730042e+04,9.045179e+04,9.437377e+04,1.028322e+05,1.280622e+05,4.534411e+05,3.704561e+05


In [5]:
latest_df = df_6mohist.sort_values('timestamp').groupby(['symbol', 'time_period']).tail(1)
result_df = pd.merge(average_df, latest_df[['symbol', 'time_period', 'volume', 'trade_count']], on=['symbol', 'time_period'], suffixes=('_average', '_latest'))
result_df['volume_ratio'] = result_df['volume_latest'] / result_df['volume_average']
result_df['trade_count_ratio'] = result_df['trade_count_latest'] / result_df['trade_count_average']
result_df.sort_values(by='volume_ratio', ascending=False, inplace=True)

# Create the dictionary {symbol: volume_ratio}
volume_ratio_dict = result_df.head(10).set_index('symbol')['volume_ratio'].to_dict()
result_df
volume_ratio_dict

{'LFST': 35.92731440297544,
 'AEHR': 11.729924621323889,
 'FIVE': 11.900549623543819}

In [6]:
result_df

,symbol,time_period,volume_average,trade_count_average,volume_latest,trade_count_latest,volume_ratio,trade_count_ratio
3238,LFST,11:30:00,104766.890110,635.098901,3763993.0,1245.0,35.927314,1.960325
213,AEHR,11:00:00,75580.791209,893.219780,1579826.0,11984.0,20.902480,13.416631
2255,FIVE,10:00:00,115832.208791,2031.626374,2308020.0,26810.0,19.925546,13.196324
2254,FIVE,09:30:00,152169.758242,2048.813187,2652792.0,21325.0,17.433109,10.408465
2256,FIVE,10:30:00,96272.120879,1818.494505,1514124.0,15313.0,15.727544,8.420702
...,...,...,...,...,...,...,...,...
4847,SHCR,11:00:00,160854.516484,434.395604,7275.0,58.0,0.045227,0.133519
2299,FREE,11:00:00,24926.362637,68.912088,1039.0,22.0,0.041683,0.319247
2304,FREE,13:30:00,12470.666667,49.344444,474.0,6.0,0.038009,0.121594
2298,FREE,10:30:00,20431.460674,65.494382,765.0,12.0,0.037442,0.183222


## Testing utils

In [3]:
alpaca.update_database()

2024-07-26 01:28:57,880 - root - INFO - Successfully read stocks_db.csv from GitHub
2024-07-26 01:28:59,693 - root - INFO - Starting concurrent download of min data across 791 tickers...
2024-07-26 01:32:00,501 - root - INFO - Done downloading min data. 1002712 rows collected.
2024-07-26 01:32:06,626 - root - INFO - Calculating avg volume and trades for 791 tickers
2024-07-26 01:32:06,767 - root - INFO -         trade_count_093000  trade_count_100000  trade_count_103000  \
symbol                                                               
AA             5854.582418         4503.637363         3789.120879   
AAL           12800.549451         9016.043956         7110.197802   
AAPL          97107.736264        67222.318681        99257.076923   
ABBV           7023.241758         6425.098901         5194.318681   
ABEV           1871.186813         1551.516484         1541.659341   

        trade_count_110000  trade_count_113000  trade_count_120000  \
symbol                         

UnboundLocalError: local variable 'updated_db' referenced before assignment

In [3]:
alpaca.get_current_positions_df()

,symbol,strat,days,value,pl%
0,IRM,BuyStreaksAt11,37,6.6,16
1,FICO,BuyStreaksAt11,29,3.2,12
2,HURN,BuyStreaksAt11,37,6.3,9
3,UTHR,BuyStreaksAt11,29,4.0,9
4,LNG,BuyStreaksAt11,20,3.6,5
5,ENLC,BuyStreaksAt11,20,3.6,3
6,T,BuyStreaksAt11,21,2.7,2
7,USOI,BuyStreaksAt11,29,3.8,1
8,PPC,BuyStreaksAt11,21,2.6,0


In [2]:
utils_for_alpaca.num_of_positions_ok(os.environ['ALPACA_API_KEY_PAPER'],os.environ['ALPACA_API_SECRET_PAPER'])

2024-06-30 23:44:36,595 - root - INFO - Currently there 33 open positions, while 15 are max allowed


False

In [3]:
alpaca.get_day_summary()

2024-07-18 01:16:41,052 - root - INFO - Summary for last 24 hours ending today on Thursday, 18.Jul:


Balance change: -259.0


Current positions:
                <html>
                <head></head>
                <body>
                    <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>symbol</th>
      <th>strat</th>
      <th>days</th>
      <th>value</th>
      <th>pl%</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>IRM</td>
      <td>BuyStreaksAt11</td>
      <td>37</td>
      <td>6.6</td>
      <td>16</td>
    </tr>
    <tr>
      <td>FICO</td>
      <td>BuyStreaksAt11</td>
      <td>29</td>
      <td>3.2</td>
      <td>12</td>
    </tr>
    <tr>
      <td>HURN</td>
      <td>BuyStreaksAt11</td>
      <td>37</td>
      <td>6.3</td>
      <td>9</td>
    </tr>
    <tr>
      <td>UTHR</td>
      <td>BuyStreaksAt11</td>
      <td>29</td>
      <td>4.0</td>
      <td>9</td>
    </tr>
    <tr>
      <td>LNG</td>
      <td>BuyStreaks

'sent-ok'

In [ ]:
alpaca.quotes('AAPL')

In [ ]:
alpaca.buypower()

In [ ]:
positions = alpaca.trading_client.get_all_positions()
positions

In [ ]:
positions[2]

In [ ]:
status = alpaca.check_trading_day()
status['market_status']

In [ ]:
alpaca.get_strategy_universe()

In [ ]:
strat_buypower = alpaca.strategy_buypower()
strat_buypower

In [ ]:
# dictio={"AAPL": 2.4e12, "GOOG": 1.8e12, "MSFT": 2.2e12, "AMZN": 1.6e12}
dictio={"AAPL": '0..3', "GOOG": 0.3, "MSFT": 0.2, "AMZN": 0.1}
# dictio={"AAPL": 0.05, "GOOG": -0.02, "MSFT": 0.01, "AMZN": -0.01}
# dictio= {"AAPL": -0.02, "GOOG": -0.02, "MSFT": 0.01, "AMZN": -0.01, }
tickers = list(dictio.keys())
investments = alpaca.allocate_capital(strat_buypower, method = 'manual', manual_weights=dictio)
# investments = alpaca.allocate_capital(strat_buypower, tickers=tickers)
# investments = alpaca.allocate_capital(strat_buypower, tickers=tickers, method = 'random')
investments

In [ ]:
df = alpaca.get_tkrs_snapshot_df(alpaca.get_ok_alpaca_stocks())

In [ ]:
df.sort_values(by='POST', ascending=False).head(5).symbol.unique().tolist()

In [ ]:
df

In [ ]:
alpaca.get_ok_alpaca_stocks()

In [ ]:
alpaca.get_tkrs_snapshot_df(Universes.Spiders)

In [ ]:
data = alpaca.get_history(symbols=Universes.Spiders,periods=570) # 570 = 15min intervals 26 intervals per day = 1 month of data
data

In [ ]:
df = alpaca.add_columns(data)
df

In [ ]:
df.to_excel('testdata.xlsx')

In [ ]:
df_day = alpaca.get_history(symbols=Universes.Spiders[:2],periods=5, FrameLength=1, frame = 'day')
df_1min = alpaca.get_history(symbols=Universes.Spiders[:2],periods=5, FrameLength=1)
df_15min = alpaca.get_history(symbols=Universes.Spiders[:2],periods=5)

In [ ]:
df_day

In [ ]:
df = alpaca.add_columns(df_day)
df

## Steps of a strategy

In [ ]:
alpaca = utils_for_alpaca.MyAlpaca(os.environ['ALPACA_API_KEY_PAPER'],os.environ['ALPACA_API_SECRET_PAPER'],'BuyTopPremarket')


In [ ]:

amount_to_invest = alpaca.strategy_buypower()

tickers_universe = alpaca.get_strategy_universe()
snapshot_df = alpaca.get_tkrs_snapshot_df(tickers_universe)
column_focus = 'POST'
tickers_to_buy_df = snapshot_df.sort_values(by=column_focus, ascending=False).head(alpaca.stocks_per_run)
tickers_to_buy_list = tickers_to_buy_df.symbol.unique().tolist()
tickers_to_buy_dict = utils_for_alpaca.create_dict_from_dataframe(tickers_to_buy_df,column_focus)
# investments = alpaca.allocate_capital(amount_to_invest, tickers=tickers_to_buy)
investments = alpaca.allocate_capital(amount_to_invest, method = 'manual', manual_weights=tickers_to_buy_dict)
alpaca.open_selected_positions(investments)

In [ ]:
alpaca = utils_for_alpaca.MyAlpaca(os.environ['ALPACA_API_KEY_PAPER'],os.environ['ALPACA_API_SECRET_PAPER'],'BuyStreaksAt11')
if alpaca.is_positive():
    amount_to_invest = alpaca.strategy_buypower()
    tickers_universe = alpaca.get_strategy_universe()
    df_day = alpaca.get_history(symbols=tickers_universe,periods=10, FrameLength=1, frame = 'day')
    df = alpaca.add_columns(df_day)
    latest_date = df['timestamp'].max()
    latest_date_rows = df[df['timestamp'] == latest_date].copy()
    max_streak = latest_date_rows['streak'].max()
    tickers_to_buy_prelim = latest_date_rows[latest_date_rows['streak'] == max_streak].symbol.unique().tolist()
    tickers_to_buy = alpaca.is_positive(tickers_to_buy_prelim) # 2nd filter - which are up today
    investments = alpaca.allocate_capital(amount_to_invest, tickers=tickers_to_buy)
    alpaca.open_selected_positions(investments)
